Part 2 data pull and evaluation at the end of this notebook

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/p

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-14 14:35:18--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.38MB/s    in 0.2s    

2023-03-14 14:35:19 (5.38 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
# https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz

from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-MM-dd")
df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...|2015-08-31 00:00:00|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...| 

In [5]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [6]:
# Get the number of rows in the DataFrame.
print(f'number of rows in the Data Frame = {df.count()}')

number of rows in the Data Frame = 2643619


In [7]:
# Drop null values, by default is any which means will drop any rows which has a NA
print((df.count(),len(df.columns)))
df2 = df.dropna()
df3 = df2.dropDuplicates()
print((df3.count(),len(df3.columns)))
df3.show()


(2643619, 15)
(2643241, 15)
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   26879429|R1015HJHGZUJVG|B00076HSUC|     421502374|Cholodin-FEL For ...|    Pet Products|          5|            1|          1|   N|                Y|      Helps Old Cats|Cholodin is a gre...|2013-08-11 00:00:00|
|         US|   49345442|R1023YMO9XR93S|B00171IVHA|      251

In [8]:
df3.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

# Transform the Data

## Create the "review_id_table".

In [9]:
# from pyspark.sql.functions import to_date
from pyspark.sql.functions import *
from pyspark.sql.types import *
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_table_df = df3.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table_df2 = review_id_table_df.withColumn("review_date",to_date(col("review_date")))
# review_id_table_df.show(truncate=False)

In [10]:
type(review_id_table_df2)

pyspark.sql.dataframe.DataFrame

In [11]:
review_id_table_df2.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [12]:
review_id_table_df2.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1015HJHGZUJVG|   26879429|B00076HSUC|     421502374| 2013-08-11|
|R1023YMO9XR93S|   49345442|B00171IVHA|      25146499| 2014-12-02|
|R10267J8PBWEVQ|   48576217|B0024E0QHC|     857059897| 2014-06-25|
|R103326H6Z3CTG|   34624253|B000UZNLGA|     144491764| 2011-11-12|
| R1035J76P280Y|   48425022|B003IRJTCC|     996646406| 2010-12-22|
| R103IP0FCBH8I|   14356933|B0002AQPKW|     152333311| 2011-12-12|
|R103WDQ4IDVIUV|   14302088|B00E8LYJS6|     394306248| 2014-12-15|
|R103WQ2GZQLH6T|   13853268|B006MUQO0M|      84171195| 2015-06-26|
|R104K30IR1GWUD|   27083213|B008V68GKG|     915709880| 2013-07-18|
|R104LLIZQ8O3DW|   18411324|B00ETS9LZ4|     259591253| 2014-06-05|
|R104OCMU5PF85T|    2654230|B008CO4H8C|     648340511| 2015-03-03|
|R104QASUJNTTNP|   21635417|B0056J2PWG|      26863086| 2014-04

## Create the "products" Table

In [13]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 

products_df2 = df3.select(["product_id", "product_title"])
products_df = products_df2.dropDuplicates()
print((products_df.count(),len(products_df.columns)))
products_df.show()

(239307, 2)
+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00134HSYS|Special Edition P...|
|B00BS6NPBG|High Tech Pet 6-P...|
|B000F930FS|PetSafe ScoopFree...|
|B000FJ9QTW|Insight ActiviToy...|
|B000ALY0OQ|SmartCat Bootsie'...|
|B00Q8ETIZ0|Dogloveit Rubber ...|
|B002VU2BA4|Kragonfly Interch...|
|B00QA3K3QM|Attmu Retractable...|
|B005DGHUC2|Zoo Med Laborator...|
|B000XY7C7C|All Four Paws, Th...|
|B004UUE26O|Animal Planet PET...|
|B008APML2C|Chuckit Medium Ul...|
|B00ZJN7T8E|1 Half Portion Ja...|
|B00NRZC8LY|Hide-A-Toy Hallow...|
|B001P3NU4E|Virbac C.E.T. Enz...|
|B003TEQ2U6|Jolly Pets Jolly ...|
|B00DJSNF0M|Multipet Lenny th...|
|B00FXVFEQG|Bags on Board Dur...|
|B005ORDWYA|Just One Bite II ...|
|B003E77OG4|  Petmate Sky Kennel|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [14]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 

from pyspark.sql.functions import desc,count

customers_df= df3.groupBy('customer_id').agg(count("customer_id").alias("customer_count"))
customers_df.show()
# df4 = df3.groupBy('customer_id').agg({'customer_id':'count'}).orderBy(desc('count(customer_id)'))
# df5 = df4.withColumnRenamed("count(customer_id)","customer_count")
# customers_df = df5.withColumn("customer_count",to_int(col("customer_count")))
# customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   21105356|             5|
|   29425552|             2|
|   17041448|             5|
|   37882821|             1|
|   42532905|             1|
|   18759528|             1|
|   19172324|             4|
|   51474275|             2|
|   27911347|             9|
|   22602819|             2|
|    5658913|            34|
|   14691760|             2|
|   13645331|             1|
|   14139374|             1|
|   14397122|             4|
|   10850500|             1|
|   47233188|             6|
|   24215961|             1|
|   16468643|             1|
|   18630443|             8|
+-----------+--------------+
only showing top 20 rows



In [15]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



## Create the "vine_table".

In [16]:
df3.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [17]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df3.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1015HJHGZUJVG|          5|            1|          1|   N|
|R1023YMO9XR93S|          2|            0|          0|   N|
|R10267J8PBWEVQ|          5|            0|          0|   N|
|R103326H6Z3CTG|          5|            1|          1|   N|
| R1035J76P280Y|          5|            6|          7|   N|
| R103IP0FCBH8I|          5|            1|          1|   N|
|R103WDQ4IDVIUV|          5|            0|          0|   N|
|R103WQ2GZQLH6T|          5|            0|          0|   N|
|R104K30IR1GWUD|          5|            0|          0|   N|
|R104LLIZQ8O3DW|          5|            0|          0|   N|
|R104OCMU5PF85T|          2|           25|         25|   N|
|R104QASUJNTTNP|          5|            0|          0|   N|
|R1058KF5BARAYI|          5|            0|          0|   N|
|R105G5FW41EYXT|          3|            

# Load

In [30]:
# Configure settings for RDS

mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cailygz0tqjt.us-east-1.rds.amazonaws.com:5432/my_data_class_db_2"
config = {"user":"postgres", "password": "123postgres456", "driver":"org.postgresql.Driver"}

In [31]:
# Write review_id_df to table in RDS
review_id_table_df2.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [32]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [33]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [34]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

PART 2

In [37]:
# rerun connection info if needed
jdbc_url="jdbc:postgresql://mypostgresdb.cailygz0tqjt.us-east-1.rds.amazonaws.com:5432/my_data_class_db_2"
config = {"user":"postgres", "password": "123postgres456", "driver":"org.postgresql.Driver"}

In [38]:
# read vine_table from AWS
vine_df = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1015HJHGZUJVG|          5|            1|          1|   N|
|R1023YMO9XR93S|          2|            0|          0|   N|
|R10267J8PBWEVQ|          5|            0|          0|   N|
|R103326H6Z3CTG|          5|            1|          1|   N|
| R1035J76P280Y|          5|            6|          7|   N|
| R103IP0FCBH8I|          5|            1|          1|   N|
|R103WDQ4IDVIUV|          5|            0|          0|   N|
|R103WQ2GZQLH6T|          5|            0|          0|   N|
|R104K30IR1GWUD|          5|            0|          0|   N|
|R104LLIZQ8O3DW|          5|            0|          0|   N|
|R104OCMU5PF85T|          2|           25|         25|   N|
|R104QASUJNTTNP|          5|            0|          0|   N|
|R1058KF5BARAYI|          5|            0|          0|   N|
|R105G5FW41EYXT|          3|            

In [39]:
type(vine_df)

pyspark.sql.dataframe.DataFrame

In [40]:

from pyspark.sql.functions import desc,count,avg,round

# evaluate all results for average star rating, total votes, helpful votes and a ratio of helpful to total votes
# compare results from vine reviewers vs non-vine reviewers

vine_df2 = vine_df.groupBy('vine').agg(round(avg("star_rating"),2).alias('avg_star_rating'),round(avg("total_votes"),2).alias('avg_total_votes'),round(avg("helpful_votes"),2).alias('avg_helpful_votes'),(round(avg('helpful_votes')/avg('total_votes')*100,2)).alias('helpful_vote_ratio'))
vine_df2.show()

+----+---------------+---------------+-----------------+------------------+
|vine|avg_star_rating|avg_total_votes|avg_helpful_votes|helpful_vote_ratio|
+----+---------------+---------------+-----------------+------------------+
|   Y|           4.07|           2.34|             1.96|             83.77|
|   N|           4.14|           1.94|             1.65|             85.26|
+----+---------------+---------------+-----------------+------------------+



In [48]:
# evaluate only results where votes were entered for average star rating, total votes, helpful votes and a ratio of helpful to total votes
# compare results from vine reviewers vs non-vine reviewers

vine_df3 = vine_df.filter(vine_df['total_votes']>0).groupBy('vine').agg(round(count("star_rating"),2).alias('n_users'),round(avg("star_rating"),2).alias('avg_star_rating'),round(avg("total_votes"),2).alias('avg_total_votes'),round(avg("helpful_votes"),2).alias('avg_helpful_votes'),(round(avg('helpful_votes')/avg('total_votes')*100,2)).alias('helpful_vote_ratio'))
vine_df3.show()


+----+-------+---------------+---------------+-----------------+------------------+
|vine|n_users|avg_star_rating|avg_total_votes|avg_helpful_votes|helpful_vote_ratio|
+----+-------+---------------+---------------+-----------------+------------------+
|   Y|   5026|           3.95|           4.76|             3.99|             83.77|
|   N| 974146|           3.83|           5.23|             4.46|             85.26|
+----+-------+---------------+---------------+-----------------+------------------+



In [49]:
vine_df5 = vine_df.filter(vine_df['total_votes']>100).groupBy('vine').agg(round(count("star_rating"),2).alias('n_users'),round(avg("star_rating"),2).alias('avg_star_rating'),round(avg("total_votes"),2).alias('avg_total_votes'),round(avg("helpful_votes"),2).alias('avg_helpful_votes'),(round(avg('helpful_votes')/avg('total_votes')*100,2)).alias('helpful_vote_ratio'))
vine_df5.show()

+----+-------+---------------+---------------+-----------------+------------------+
|vine|n_users|avg_star_rating|avg_total_votes|avg_helpful_votes|helpful_vote_ratio|
+----+-------+---------------+---------------+-----------------+------------------+
|   Y|     27|           3.93|          216.3|            206.7|             95.57|
|   N|   4561|            3.9|          236.7|           223.55|             94.44|
+----+-------+---------------+---------------+-----------------+------------------+



In [50]:
vine_df6 = vine_df.filter(vine_df['total_votes']>1000).groupBy('vine').agg(round(count("star_rating"),2).alias('n_users'),round(avg("star_rating"),2).alias('avg_star_rating'),round(avg("total_votes"),2).alias('avg_total_votes'),round(avg("helpful_votes"),2).alias('avg_helpful_votes'),(round(avg('helpful_votes')/avg('total_votes')*100,2)).alias('helpful_vote_ratio'))
vine_df6.show()

+----+-------+---------------+---------------+-----------------+------------------+
|vine|n_users|avg_star_rating|avg_total_votes|avg_helpful_votes|helpful_vote_ratio|
+----+-------+---------------+---------------+-----------------+------------------+
|   Y|      1|            4.0|         1277.0|           1239.0|             97.02|
|   N|     77|           3.96|        1619.64|          1562.19|             96.45|
+----+-------+---------------+---------------+-----------------+------------------+



In [55]:
# check to see if there is difference in results when grouped by total votes i.e. >1, >100, >1000
from pyspark.sql.functions import desc,count,avg,round,col

# https://stackoverflow.com/questions/51063624/whats-the-equivalent-of-pandas-value-counts-in-pyspark
vine_df4 = vine_df.filter(vine_df['total_votes']>1).groupBy('total_votes').count().orderBy(col('count').desc())
vine_df4.show()

+-----------+------+
|total_votes| count|
+-----------+------+
|          2|180568|
|          3| 92799|
|          4| 57560|
|          5| 37739|
|          6| 26695|
|          7| 20084|
|          8| 15410|
|          9| 12034|
|         10|  9827|
|         11|  7959|
|         12|  6803|
|         13|  5783|
|         14|  4824|
|         15|  4233|
|         16|  3694|
|         17|  3284|
|         18|  2885|
|         19|  2624|
|         20|  2377|
|         21|  2033|
+-----------+------+
only showing top 20 rows



In [56]:
print((vine_df4.count(),len(vine_df4.columns)))

(732, 2)
